In [1]:
import pandas as pd
import os
import numpy as np
import glob
from netCDF4 import Dataset,date2num,num2date
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import xlsxwriter

In [4]:
ignorCty=['Nậm Păm']
ignorTB=['Mường Khoa','Sốp Cộp','Mường Trai','Cao Phong','Tu Lý','Mường Chiềng','Yên Thủy','Ban Hàng Đồi','Bao La','Tân Lạc']
ignorVB=['Phú Bình','Xuân Giang 1','Vĩnh Tường','Lăng Can 1','Đình Cả','Đại Từ', 'Đạo Viện 1','Thanh Thủy','Tam Dương',
         'Phúc Yên','Phấn Mễ','Quản Bạ','Việt Lâm','Ký Phú', 'Phố Đu', 'Yên Bình','Kim Quan', 'Điềm Mặc 1','Nậm Ty','Phủ Thông',
         'Mèo Vạc','Vũ Chấn','Bình Xuyên','Phương Viên','La Hiên', 'Minh Quang', 'Yên Lạc','Phổ Yên','Sơn Nam', 
         'Linh Hồ','Vĩ Thượng','Yên Minh 1','Kỳ Lâm','Việt Vinh','Xuân Hoà', 'Yên Lập','Trạm Tấu','Thọ Sơn (Đoan Hùng)',
         'Thạch Kiệt 1','Lâm Thao','Kiên Lao','Thanh Ba','Ba Khe','Cẩm Khê','Hạ Hoà','Thanh Thủy (La Phù)','Mỹ Lương']
name_stations_Cty={'Trạm Đo':pd.Series(['Chiềng Ơn','Nậm Păm','Nậm Giôn','Chiềng Khay','Nậm Hàng 2','Nậm Sỏ','Nậm Hàng 1',
                                        'Mường Mùn','Mường Mít','Ta Ma', 'Phình Sáng', 'Thị Trấn Tân Uyên','Khun Há', 
                                        'Bản Lang','Nậm Hăn 2','Nậm Ét','Sín Chải','Nậm Ban','Mường Toong','Mù Cả'])}

ra={'Ranges':[19.16,31.40,35.36,77.11,58.14,91.24,23.42,61.53,17.72,24.64,71.79,78.41,114.99,54.56,23.07,59.96,
                      101.22,118.42,149.36,22.09,105.70,120.21,61.58,165.02,92.60,115.12,128.15,151.47,95.39,111.60,173.22,31.77,117.85,15.22,81.76,
                     130.77,137.89,71.50,84.79,109.30,146.26,172.84,156.25,109.23,139.88,124.58,132.75,106.05,126.36,107.87,94.85,147.45,70.73,189.38,
                     114.62,131.83,112.25,122.11,83.91,93.70,114.29,121.50]}
name_stations_Cty_new={'Trạm Đo':pd.Series(['Nậm Sỏ','Nậm Hàng 1','Mường Mùn','Phình Sáng','Nậm Ban','Chiềng Ơn','Nậm Hàng 2','Mường Mít',
                                           'Thị Trấn Tân Uyên','Bản Lang','Sín Chải','Chiềng Khay','Nậm Ét','Mường Toong','Nậm Păm','Nậm Giôn',
                                            'Ta Ma','Khun Há','Nậm Hăn 2','Mù Cả'])}
ra_new={'Ranges':[58.14,91.24,23.42,24.64,101.22,19.16,77.11,61.53,71.79,114.99,59.96,35.36,23.07,118.42,31.40,17.72,78.41,54.56,149.36,
                 22.09,105.70,120.21,61.58,165.02,92.60,115.12,128.15,151.47,95.39,111.60,173.22,31.77,117.85,15.22,81.76,
                     130.77,137.89,71.50,84.79,109.30,146.26,172.84,156.25,109.23,139.88,124.58,132.75,106.05,126.36,107.87,94.85,147.45,70.73,189.38,
                     114.62,131.83,112.25,122.11,83.91,93.70,114.29,121.50]}

In [14]:
#Gộp các files gauge/loại bỏ một số trạm/tạo file có lượng mưa của các trường hợp
Cases=['General']#'Full stratiform','Squall line'
for case in Cases:
    a=len(next(os.walk('D:/gauge datas 2022/'+case+'/'))[1]) # a là số events có trong folder
    zz=[]
    sheets=[]
    for r in range(1,16,1):
        for file in os.listdir('D:/gauge datas 2022/'+case+'/'+str(r)+'/'):
            if file[19:20]=='C':
                df_Cty=pd.read_excel('D:/gauge datas 2022/'+case+'/'+str(r)+'/'+file,skiprows=(0,1,2))
                df_Cty['Trạm Đo']=name_stations_Cty_new['Trạm Đo']
                df_Cty=df_Cty[~df_Cty['Trạm Đo'].isin(ignorCty)]
                del df_Cty['Tổng']
            if file[36:37]=='T':
                df_TB=pd.read_excel('D:/gauge datas 2022/'+case+'/'+str(r)+'/'+file,skiprows=(0,1,2))
                df_TB=df_TB[~df_TB['Trạm Đo'].isin(ignorTB)]
                del df_TB['Tổng']
            if file[36:37]=='V':
                df_VB=pd.read_excel('D:/gauge datas 2022/'+case+'/'+str(r)+'/'+file,skiprows=(0,1,2))
                df_VB=df_VB[~df_VB['Trạm Đo'].isin(ignorVB)]
                del df_VB['Tổng']
        df_Cty_TB=df_Cty.append([df_TB],ignore_index = True )
        df=df_Cty_TB.append([df_VB],ignore_index = True )
        df.insert(0,'Ranges',ra_new['Ranges'])
        df=df.sort_values("Ranges",ignore_index=True)
        df=df.drop(df.columns[[-1]], axis=1)
        df=df.drop([56,57,58,59,60,61]) # ignor gauges with R>150km #CC low 16,24,25,34,40,45,46,53,54,
        df=df.drop(df.columns[[1]], axis=1)
        zz.append(df)
        sheets.append(str(r)) 
    out_path = r"D:/data/Gauges/Finall/General/"+case+".xlsx"
    writer = pd.ExcelWriter(out_path,engine='xlsxwriter')   
    for dataframe, sheet in zip(zz, sheets):
        dataframe.to_excel(writer, sheet_name=sheet, startrow=0 , startcol=0)   
    writer.save()

In [2]:
#Sử lí file gauge datas thủy điện mới
Cases=['General']#'Full stratiform','Squall line'
for case in Cases:
    a=len(next(os.walk('D:/gauge datas 2022/'+case+'/'))[1]) # a là số events có trong folder
    zz=[]
    sheets=[]
    for r in range(1,a+1,1):
        for file in os.listdir('D:/gauge datas 2022/'+case+'/'+str(r)+'/'):
            if file[19:20]=='C':
                df_Cty=pd.read_excel('D:/gauge datas 2022/'+case+'/'+str(r)+'/'+file)
                #df_Cty['Trạm Đo']=name_stations_Cty['Trạm Đo']
                #df_Cty=df_Cty[~df_Cty['Trạm Đo'].isin(ignorCty)]
                df_Cty=df_Cty.drop([3,4,5,6,10,11,13,14,15,16,17,18,19,20,21,23,24,25,26,28,30,31,32,33,35,39,40,41,42,44,45,46,47,52,53,57,58,59,61])
                df_Cty.to_excel("D:/gauge datas 2022/General/"+str(r)+'/'+file)

KeyError: '[23 24 25 26 28 30 31 32 33 35 39 40 41 42 44 45 46 47 52 53 57 58 59 61] not found in axis'

In [3]:
#Xóa số liệu 1 trạm
Case='General.xlsx'
#Choose events of a case
Case_dic={Case:[str(i) for i in[*range(1,51,1)]]}
data_frame=[]
number_sheets=[]
for r in Case_dic[Case]:
    #make gauges dataframe at each events
    df_gauges=pd.read_excel('E:/data/Gauges/Finall/'+Case,sheet_name=r,index_col=0)
    df_gauges=df_gauges.drop([8,10,16,21,22,24,26,29,30,37,39,41,44,45,46,49,50,53])
    data_frame.append(df_gauges)
    number_sheets.append(str(r))
out_path = r"E:/data/Gauges/Finall/PA6/"+Case[:-5]+".xlsx"
writer = pd.ExcelWriter(out_path,engine='xlsxwriter')   
for dataframe, sheet in zip(data_frame, number_sheets):
    dataframe.to_excel(writer, sheet_name=sheet, startrow=0 , startcol=0)   
writer.save()    

In [4]:
df_gauges

,Unnamed: 0,Ranges,Trạm Đo,16:10 05/04,16:20 05/04,16:30 05/04,16:40 05/04,16:50 05/04,17:00 05/04,17:10 05/04,...,18:30 05/04,18:40 05/04,18:50 05/04,19:00 05/04,19:10 05/04,19:20 05/04,19:30 05/04,19:40 05/04,19:50 05/04,20:00 05/04
0,0,15.22,Mường Sại,0.0,0.0,0.0,1.0,1.0,0.8,0.8,...,0.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,17.72,Ta Ma,0.0,0.0,0.0,0.0,0.4,0.4,0.2,...,0.6,0.4,1.6,2.2,2.6,3.4,6.0,8.4,20.8,17.2
2,2,19.16,Chiềng Ơn,0.0,0.0,0.2,0.8,1.0,0.6,0.6,...,0.8,0.4,2.6,0.2,0.0,0.0,0.0,0.0,0.0,0.0
3,3,22.09,Thuận Châu,0.0,0.0,0.0,0.0,0.2,0.0,0.2,...,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,23.07,Nậm Ét,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,5,23.42,Mường Mùn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,8.8,9.0,9.2,2.4,0.4,1.0,1.8,2.2,2.0
6,6,24.64,Phình Sáng,0.0,0.0,0.0,0.0,0.0,0.6,0.2,...,0.0,5.6,3.6,2.8,9.8,13.0,9.6,4.8,2.8,3.6
7,7,31.40,Nậm Giôn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,8,31.77,Mường Ảng,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,14.4,7.6,4.4,2.8,2.0,0.8,0.0,0.0,0.0,2.0
9,9,35.36,Chiềng Khay,0.0,0.0,0.0,0.0,0.2,0.4,0.8,...,6.6,4.0,0.2,0.4,1.4,1.2,1.0,1.0,0.8,2.2
